## Comment contrôler 6 LED avec 3 fils de commande seulement ?

Dans la vie courante on est habitué à contrôler une ampoule avec deux fils, nous avons vu qu'en branchant deux LEDs symétriquement il était possible en permuttant la valeur des broches (0 ou 1)d'obtenir un clignotement. Il existe aussi des dispositifs présentant de nombreuses LEDs contrôlées individuellement ou par groupes, il s'agit par exemple des chenillards lumineux dans les espaces ludiques ou des guirlandes de Noël. Malgré le grand nombre de LEDs ces dispositifs sont contrôlés par un très faible nombre de fils, comment cela est-il possible ?

# Principe
Nous allons aborder un exemple basique permettant de contrôler 6 LED avec 3 fils, le procédé utilisé est le Charlie-Plexing. Ce procédé permet de contrôler avec n fils `n * (n-1)` LEDs; soit pour nous 3 x 2 = 6.
On utilise pour cette partie le montage encadré en rouge ci-dessous.

# Montage

![CharliePlexing à 6 LED](images/ledboard6.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Les fils gris représentent les points de branchement des 3 fils de contrôle, le cablage est représenté par les fils rouges. Ce procédé repose sur la possibilité de mettre la broche non pas dans deux mais dans 3 états : 
 * état masse ..............valeur out 0
 * état alimentation........valeur out 1
 * état résistance infinie .valeur in Z
 
Le Schéma de principe du fonctionnement est donné dans l'image suivante :

![Schéma de principe du CharliePlexing à 6 LED](images/anim6led.gif)
                        ![licence : Gérard Vidal](images/Licence.jpg)               

Nous représenterons chaque état par la séquence d'abbréviations la caractérisant dans l'ordre des broches 1-2-3. Par exemple 0Z1 signifie que le + est sur la broche 3 que le - est sur la broche 1 et que la broche 2 est bloquée. On a donc les états suivants
 * 10Z
 * 1Z0
 * 01Z
 * 0Z1
 * Z10
 * Z01
 
 # Programme
 
Cela fournit bien 6 états possibles pour notre circuit à 3 fils. Nous allons **définir** (instruction *def*) un ensemble d'instructions permettant en fonction de l'ordre dans lequel sont passées les broches de réaliser l'état attendu. Il s'agit d'une  *fonction* (définie par la commande **def**)que nous nommerons du nom de l'état dans l'ordre des 3 broches [0,1,z] du circuit qu'elles provoquent, nous définissons aussi une *fonction* qui éteint toutes les leds. 

La figure ci-dessous présente l'état du branchement effectivement réalisé pour réaliser le chenillard et l'arc en ciel.

![Détail CharliePlexing à 6 LED et diode RGB](images/detailchen+gb.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)


In [1]:
import mraa, time

In [2]:
led1 = mraa.Gpio(31) # extérieur du dispositif
led2 = mraa.Gpio(32) # milieu
led3 = mraa.Gpio(33) # intérieur

In [3]:
dur = 0.04

In [4]:
def etat_01Z(a,b,c) :
    a.dir(mraa.DIR_OUT)
    b.dir(mraa.DIR_OUT)
    c.dir(mraa.DIR_IN)
    a.write(0)
    b.write(1)
    time.sleep(dur)
    return

def etat_000(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_IN)
    return

Nous définisons enfin une dernière fonction qui décrit l'enchainement des états des broches pour réaliser le chenillard' une 7ème qui assurera l'enchaînement des états blink6.

In [12]:
def blink6(leda,ledb,ledc) :
    for i in range(100) :
        etat_01Z(leda,ledb,ledc)
        etat_01Z(ledb,leda,ledc)
        etat_01Z(leda,ledc,ledb)
        etat_01Z(ledc,leda,ledb)
        etat_01Z(ledb,ledc,leda)
        etat_01Z(ledc,ledb,leda)
        etat_000(leda,ledb,ledc)
        etat_01Z(ledc,ledb,leda)
        etat_01Z(ledb,ledc,leda)
        etat_01Z(ledc,leda,ledb)
        etat_01Z(leda,ledc,ledb)
        etat_01Z(ledb,leda,ledc)
        etat_01Z(leda,ledb,ledc)
        etat_000(leda,ledb,ledc)
    etat_000(leda,ledb,ledc)
    return

L'exécution est lancée ici :

In [13]:
blink6(led1,led2,led3)

## Création d'un arc-en-ciel avec une LED RGB

La Led RGB permet de mélanger les couleurs comme cela se passe dans un téléviseur à LED. Chaque point de l'écran crée la couleur requise en mélangeant les trois couleurs Rouge Vert Bleu dans les proportions adéquates. Une nouvelle bibliothèque est ajoutée afin de déyecter les entrées au clavier pendant l'excution des tâches.

In [39]:
import RPi.GPIO as gpio, time
gpio.setmode(gpio.BOARD)  # choisir BCM ou BOARD pour identifier les broches. Ici BOARD
import _thread as thread

gpio.setup(15, gpio.OUT)# GPIO 22 broche 15 output  
gpio.setup(16, gpio.OUT)# GPIO 23 broche 16 output 
gpio.setup(18, gpio.OUT)# GPIO 24 broche 18 output  

pb = gpio.PWM(15, 200)    # led bleue crée un objet pg  PWM sur le port GPIO22 à 200 Hertz  T=5ms 
pg = gpio.PWM(16, 200)    # led verte crée un objet pb  PWM sur le port GPIO23 à 200 Hertz  T=5ms 
pr = gpio.PWM(18, 200)    # led rouge crée un objet pr  PWM sur le port GPIO24 à 200 Hertz  T=5ms 

AVant de traiter la question de l'arc en ciel il est nécessaire de vérifier que les branchements sont corrects en allumant successivement les led Rouge verte et bleue de la led RGB. Les led sont allumées tour à tour pendant 3 secondes à 50% de leur capacité.

In [40]:
dur = 3
pr.start(50)
time.sleep(dur)
pr.stop()

In [41]:
pg.start(50)
time.sleep(dur)
pg.stop()

In [42]:
pb.start(50)
time.sleep(dur)
pb.stop()            

In [43]:

dur = 0.15

# veille sur l'input
def input_thread(CheckInput):
    check = input()
#    CheckInput = CheckInput.append(check)
    CheckInput.append(check)
    return

def rainbow():
    CheckInput = []
    thread.start_new_thread(input_thread, (CheckInput,))
    print ('Type "stop" to stop modulation')
    
    pr.start(0)             # allumage à 0 % (0% duty cycle) 
    for i in range(0,100,5) :
        pr.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
        time.sleep(dur)
        
    while True :
        pg.start(0)             # allumage à 0 % (0% duty cycle) 
        for i in range(0,100,5) :
            pg.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
            time.sleep(dur)
        for i in range(100,0,-5) :
            pr.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
            time.sleep(dur)
        pr.stop()
        pb.start(0)             # allumage à 0 % (0% duty cycle) 
        for i in range(0,100,5) :
            pb.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
            time.sleep(dur)
        for i in range(100,0,-5) :
            pg.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
            time.sleep(dur)
        pg.stop()
        pr.start(0)             # allumage à 0 % (0% duty cycle) 
        for i in range(0,100,5) :
            pr.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
            time.sleep(dur)
        for i in range(100,0,-5) :
            pb.ChangeDutyCycle(i)   # de 0 à 100% par pas de 5% par 0.25s
            time.sleep(dur)
        pb.stop()
        
        # Sortie de boucle infinie
        if CheckInput == ['stop']  : 
            pr.stop()
            pg.stop()
            pb.stop()            
            break
    return


rainbow()


Type "stop" to stop modulation
stop


In [5]:
pr.stop()
pg.stop()
pb.stop()

In [31]:
gpio.cleanup()          #nettoyage de l'état du gpio  